## HW2 - Zhamilya Saparova


In [117]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
pip install ir_measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
import pandas as pd
import numpy as np
import ir_measures
from ir_measures import *

In [120]:
es = Elasticsearch("http://localhost:9200")
ELASTIC_PASSWORD = 'EW3FiZs3r32l9yYhvwWklUVN'

CLOUD_ID = 'IR_2_search:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRkZTFhZjVhMzYxYmU0M2ZlODkwYTZkNGJkYjE0YjYwMiQ4Y2U3NWNkZWMyOTU0NWYzYjBmZDM5ZWM5MDgyNTgwZg=='
es = Elasticsearch(cloud_id = CLOUD_ID, basic_auth=('elastic', ELASTIC_PASSWORD))

print("Connection to ES Server successful")

Connection to ES Server successful


In [121]:
es.indices.delete(index='hello')

ObjectApiResponse({'acknowledged': True})

In [122]:
es.indices.create(index='hello')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hello'})

In [123]:
settings_final = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            }
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                        'snowball_eng'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
            'filter': {
                'snowball_eng':{
                    'type': 'snowball',
                    'language': 'english'
                }
            }
        }
    }
    
}

In [124]:
settings_final_nostem = {
    'mappings': {
        'properties': {
            'id_right': {
                'type': 'integer'
            },
            'text_right': {
                'type': 'text',
                'fields': {
                    'white':{
                        'type': 'text',
                        'analyzer': 'white_analyzer'
                    }
                }
            },
        }
    },
    'settings': {
        'analysis': {
            'analyzer': {
                'white_analyzer':{
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase',
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace',
                }
            },
        }
    }
}

In [125]:
def recreate_index():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final)

In [126]:
def recreate_index_nostem():
    es.indices.delete(index='hello')
    es.indices.create(index='hello', body = settings_final_nostem)

In [127]:
def check_analyzer(analyzer, text):
    body = analyzer
    body['text'] = text
    
    tokens = es.indices.analyze(index='hello', body=body)['tokens']
    tokens = [token_info['token'] for token_info in tokens]
    return tokens

In [ ]:
# recreate_index()

In [128]:
recreate_index_nostem()

<ipython-input-126-8cc1ef7b44ce>:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='hello', body = settings_final_nostem)


## INDEX DOCUMENTS

In [129]:
doc_df = pd.read_csv('/content/documents.csv')
doc = doc_df.to_dict('records')

In [130]:
type(doc), len(doc), type(doc_df), doc_df.shape

(list, 369721, pandas.core.frame.DataFrame, (369721, 2))

In [131]:
doc_df.head()

,id_right,text_right
0,1781133,it was used in landing craft during world war ...
1,2426736,after rejecting an offer from cambridge univer...
2,2224122,mat zan coached kuala lumpur fa in 1999 and wo...
3,219642,a barcode is a machine readable optical label ...
4,1728654,since the subordination of the monarchy under ...


In [132]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [133]:
def es_actions_generator():
    for doc_id in doc:
        yield create_es_action('hello', doc_id['id_right'], doc_id)

In [142]:
# ADDING DOCS TO THE INDEX
%%time 
for ok, result in parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000):
    if not ok:
        print(result)

CPU times: user 1min 21s, sys: 497 ms, total: 1min 22s
Wall time: 1min 38s


## SEARCH


In [143]:
def search(query, *args):
    pretty_print_result(es.search(index='hello', body=query, size=20), args)
                        
def pretty_print_result(search_result, fields=[]):
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='hello', id=doc_id)['_source']

In [136]:
%%time 

query = {
    'query': {
        'match_all': {}
    }
}
search(query)

<ipython-input-135-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


Total documents: 10000
Doc 1989806, score is 1.0
Doc 666954, score is 1.0
Doc 854184, score is 1.0
Doc 2339707, score is 1.0
Doc 2277293, score is 1.0
Doc 1410326, score is 1.0
Doc 1050121, score is 1.0
Doc 1313260, score is 1.0
Doc 905119, score is 1.0
Doc 991693, score is 1.0
Doc 2159376, score is 1.0
Doc 1016236, score is 1.0
Doc 2249596, score is 1.0
Doc 1636859, score is 1.0
Doc 2062306, score is 1.0
Doc 368504, score is 1.0
Doc 2387543, score is 1.0
Doc 2432898, score is 1.0
Doc 234130, score is 1.0
Doc 495680, score is 1.0
CPU times: user 131 ms, sys: 4.19 ms, total: 135 ms
Wall time: 5.35 s


In [137]:
%%time

query = {
    'query': {
        'bool': {
            'must': {
                'match': {
                    'text_right': 'private boats'
                }
            }
        }
    }
}

search(query)

<ipython-input-135-fae58037f7d9>:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  pretty_print_result(es.search(index='hello', body=query, size=20), args)


Total documents: 10000
Doc 2355214, score is 13.976819
Doc 644717, score is 13.686264
Doc 1596321, score is 13.1172695
Doc 2412578, score is 12.66897
Doc 352785, score is 12.558475
Doc 1272069, score is 12.242398
Doc 1226233, score is 12.007725
Doc 1856656, score is 12.007725
Doc 880419, score is 12.007725
Doc 919902, score is 12.007725
Doc 1879751, score is 12.007725
Doc 1110284, score is 11.860317
Doc 1329021, score is 10.934549
Doc 2029858, score is 10.934549
Doc 1714910, score is 10.934549
Doc 1964437, score is 10.934549
Doc 551366, score is 10.934549
Doc 885487, score is 10.934549
Doc 2075473, score is 10.934549
Doc 1185710, score is 10.884844
CPU times: user 144 ms, sys: 11.2 ms, total: 155 ms
Wall time: 10 s


In [138]:
import datetime
import time
q = pd.read_csv('/content/queries.csv')
q = q.to_dict('records')
runs = dict()
for i in range(len(q)):
  st = time.time()
  queries = dict()
  query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': q[i]['text_left']
                    }
                }
            }
        }
    }
  search_results = es.search(index = 'hello', body = query, size = 20)
  res = search_results['hits']
  for hit in res['hits']:
    doc_id = str(hit['_id'])
    score = float(hit['_score'])
    query_id = i
    queries[doc_id] = score
  runs[str(q[i]['id_left'])] = queries
  et = time.time()
  elapsed_time = et - st
  print(query_id, '\t', elapsed_time)


0 	 0.03603792190551758
1 	 0.029225826263427734
2 	 0.03914070129394531
3 	 0.03237342834472656


<ipython-input-138-a68b554ae5e3>:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index = 'hello', body = query, size = 20)


4 	 0.03465890884399414
5 	 0.03167247772216797
6 	 0.04415273666381836
7 	 0.029502153396606445
8 	 0.033792734146118164
9 	 0.028650999069213867
10 	 0.03186488151550293
11 	 0.03462529182434082
12 	 0.03092789649963379
13 	 0.03533315658569336
13 	 0.02958965301513672
15 	 0.028217315673828125
16 	 0.029741287231445312
17 	 0.039865732192993164
18 	 0.03797483444213867
19 	 0.04868674278259277
20 	 0.029242515563964844
21 	 0.034291982650756836
22 	 0.04587244987487793
23 	 0.031708717346191406
24 	 0.0346677303314209
25 	 0.040158748626708984
26 	 0.030676841735839844
27 	 0.036947011947631836
28 	 0.030858516693115234
29 	 0.03109884262084961
30 	 0.036988258361816406
31 	 0.03349113464355469
32 	 0.03130054473876953
33 	 0.03111124038696289
34 	 0.038622379302978516
35 	 0.03138089179992676
36 	 0.03227567672729492
37 	 0.03508877754211426
38 	 0.03201794624328613
39 	 0.03369545936584473
40 	 0.030934572219848633
41 	 0.033418893814086914
42 	 0.031966209411621094
43 	 0.0403318

## PART 2 EVALUATION

In [139]:
#  Qrel(query_id='13540', doc_id='1276852', relevance=1, iteration='0'),

qrels = ir_measures.read_trec_qrels('/content/qrels')
qrels_ = dict()
qrels_new = dict()
for i in list(qrels):
  query_id = i[0]
  doc_id = i[1]
  relevance = i[2]
  qrels_[doc_id] = relevance
  qrels_new[query_id] = qrels_

In [140]:
import datetime
import time
q = pd.read_csv('/content/queries.csv')
q = q.to_dict('records')
runs = dict()
for i in range(len(q)):
  st = time.time()
  queries = dict()
  query = {
        'query': {
            'bool': {
                'must': {
                    'match': {
                        'text_right': q[i]['text_left']
                    }
                }
            }
        }
    }
  search_results = es.search(index = 'hello', body = query, size = 20)
  res = search_results['hits']
  for hit in res['hits']:
    doc_id = str(hit['_id'])
    score = float(hit['_score'])
    query_id = i
    queries[doc_id] = score

  runs[str(q[i]['id_left'])] = queries
  
scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, runs)
et = time.time()
elapsed_time = et - st
print(scores)
print('TIME', elapsed_time)

<ipython-input-140-858307cd7779>:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index = 'hello', body = query, size = 20)


{P@10: 0.22599999999999987, AP: 0.0004083007253942143, P@20: 0.16450000000000006}
TIME 0.17934083938598633


In [141]:
# [ScoredDoc(query_id='158491', doc_id='625257', score=15.660703104969318),

bm25 = ir_measures.read_trec_run('/content/BM25.res')
bm25_ = dict()
creators_runs = dict()
for i in list(bm25):
  st = time.time()
  query_id = i[0]
  doc_id = i[1]
  score = i[2]
  bm25_[doc_id] = score
  creators_runs[query_id] = bm25_

scores = ir_measures.calc_aggregate([P(rel=1)@10, P(rel=1)@20, AP], qrels_new, creators_runs)
et = time.time()
elapsed_time = et - st
print(scores)
print('TIME', elapsed_time)

{P@10: 0.3000000000000005, AP: 0.010808749000016847, P@20: 0.3000000000000005}
TIME 0.726341724395752
